In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# pd.set_option('display.max_colwidth', None)

In [3]:
pd.reset_option('display.max_colwidth')

In [84]:
collected_08_23 = {
    "otodom_krk" : "../original_data/collected_08_23/master_data/otodom_krk.json",
    "otodom_gda" : "../original_data/collected_08_23/master_data/otodom_gda.json",
    "olx_gda" : "../original_data/collected_08_23/master_data/olx_gda.json",
    "olx_krk" : "../original_data/collected_08_23/master_data/olx_krk.json",
    "gratka_gda" : "../original_data/collected_08_23/master_data/gratka_gda.json",
    "gratak_krk" : "../original_data/collected_08_23/master_data/gratka_krk.json",
    "trojmiasto_gda" : "../original_data/collected_08_23/master_data/trojmiasto_gda.json"
}

# Create a dictionary to store DataFrames
dataframes_08_23 = {}

# Iterate through the JSON data and create DataFrames
for key, value in collected_08_23.items():
    dataframes_08_23[key] = pd.read_json(value)  # Assuming the data is in JSON format

# Now, you have a dictionary of DataFrames where each key is a dataset
# You can access them like this:
otodom_krk_df = dataframes_08_23["otodom_krk"]
otodom_gda_df = dataframes_08_23["otodom_gda"]
olx_gda_df = dataframes_08_23["olx_gda"]
olx_krk_df = dataframes_08_23["olx_krk"]
gratka_gda_df = dataframes_08_23["gratka_gda"]
gratak_krk_df = dataframes_08_23["gratak_krk"]
trojmiasto_gda_df = dataframes_08_23["trojmiasto_gda"]

In [89]:
df = olx_gda_df

In [86]:
olx_gda_df.isna().sum()

rent                    0
location                0
username                0
on_olx_since            0
last_activity           0
add_date                0
title                   0
link                    0
description             0
Prywatne               46
Poziom                 12
Umeblowane              0
Rodzaj zabudowy         0
Powierzchnia            0
Liczba pokoi            0
Czynsz (dodatkowo)      0
Firmowe               328
dtype: int64

In [43]:
import pandas as pd

# Assuming you have a DataFrame named 'df' and you want to check a specific column named 'your_column_name'
target_string = 'negocjacji'
column_name = 'rent'

# Create a boolean mask to check if each row in the specified column contains the target string
mask = olx_gda_df[column_name].str.contains(target_string, na=False)

# Count the number of True values in the mask (i.e., the number of rows containing the target string)
count = mask.sum()

print(f"The string '{target_string}' appears in {count} rows of the column '{column_name}'.")


The string 'negocjacji' appears in 42 rows of the column 'rent'.


In [102]:
olx_gda_df['location'].value_counts()

location
Śródmieście                                     78
Przymorze Wielkie                               43
Brzeźno                                         33
Przymorze Małe                                  32
Wrzeszcz                                        32
Żabianka - Wejhera - Jelitkowo - Tysiąclecia    30
Chełm z dzielnicą Gdańsk Południe               17
Jasień                                          14
Ujeścisko - Łostowice                           14
Letnica                                         12
Piecki-Migowo                                   10
Zaspa Młyniec                                   10
Orunia - Św. Wojciech - Lipce                    9
Zaspa Rozstaje                                   7
Siedlce                                          7
Stogi z Przeróbką                                6
Oliwa                                            5
Aniołki                                          5
Wzgórze Mickiewicza                              2
Matarnia              

In [44]:
olx_gda_df.shape

(374, 17)

In [66]:
def convert_numeric_to_float(convertable_string):
    return float(convertable_string.replace("zł", "").replace(" ", "").replace("\ndonegocjacji", ""))

def clear_olx(df: pd.DataFrame, city = 'Gdansk'):
    df.drop(
        labels=[
            'link', 
            'username', 
            'on_olx_since', 
            'last_activity',
            'title',
            'description'
        ],
        inplace= True,
        axis= 1
    )
    
    is_negotiable_list = []
    
    for index, row in df.iterrows():

        # negotiable
        if 'do negocjacji' in row['rent']:
            is_negotiable_list.append(True)
        else:
            is_negotiable_list.append(False)
    
    # negotiable apply
    df['negotiable'] = is_negotiable_list


    # rent
    df['rent'] = df['rent'].apply(convert_numeric_to_float)

    # rent_extra
    df = df.rename(mapper={"rent_extra": "Czynsz (dodatkowo)"}, axis=1)
    df['rent_extra'] = df['rent_extra'].apply(convert_numeric_to_float)

In [67]:
clear_olx(olx_gda_df)

[False, False, False, False, False, True, False, False, True, False, False, True, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, Fal